<a href="https://colab.research.google.com/github/nicolaiberk/bild/blob/main/code/content_analysis/bert/crime/bert/crime_annotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
import os
import pandas as pd
import csv
import sys
import numpy as np


# parameters
clsfr_path = "drive/MyDrive/Bild/crime_clsfr_BERT_torch"
input_path = "https://www.dropbox.com/s/ousyjlju18fopny/_migration_articles_BERT.csv?dl=1"
output_path = "drive/MyDrive/Bild/crime_annotations.csv"
batch_size = 16

# load classifier and tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
model = AutoModelForSequenceClassification.from_pretrained(clsfr_path)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-german-cased', model_max_length = 512)
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer, device = 0, padding='max_length', truncation=True)

Sanity Check:

In [ ]:
# Test model
text = "Die Polizei hat gestern zwei Flüchtige nach einem Bankraub festgenommen. Nach einem kurzen Schusswechsel konnten die mutmaßlichen Täter überwältigt und festegnommen werden. Die Staatsanwaltschaft ermittelt."
classifier(text)

In [ ]:
text = "Schleswig-Holstein hat einen neuen Landtag gewählt. Mit dem vorläufigen Endergebnis kommt die CDU als stärkste Kraft auf 43,4 Prozent der Stimmen, die SPD erreicht 16 Prozent, die Grünen sind zweitstärkste Kraft mit 18,3 Prozent."
classifier(text)

Looking good.

In [ ]:
mig_articles = pd.read_csv(input_path)

In [ ]:
mig_articles.head()

In [ ]:
mig_articles["crime"] = classifier([article for article in mig_articles.text])

In [ ]:
import re
int(re.match('LABEL_(\d)', 'LABEL_0').group(1))


In [ ]:
# clean
labels = []
prob = []
for dt in mig_articles.crime:
  labels.append(int(re.match('LABEL_(\d)', dt['label']).group(1)))
  prob.append(float(dt['score']))

mig_articles['crime_label'] = labels
mig_articles['crime_prob'] = prob

In [ ]:
mig_articles.to_csv(output_path)

In [ ]:
mig_articles.head()